In [1]:
from google.colab import drive

# Check if drive is already mounted
import os
if not os.path.exists('/content/drive'):
  drive.mount('/content/drive')
else:
  print("Google Drive is already mounted at /content/drive")

Mounted at /content/drive


In [2]:
!cp /content/drive/MyDrive/guessdata/brain_tumor.zip /content/

In [3]:
!ls /content/
!sudo apt-get install zip unzip #unrarı kurar

brain_tumor.zip  drive	sample_data
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
zip is already the newest version (3.0-12build2).
unzip is already the newest version (6.0-26ubuntu3.2).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [5]:

!unzip /content/brain_tumor.zip -d /content/brain_tumor

Görüntülenen çıkış son 5000 satıra kısaltıldı.
  inflating: /content/brain_tumor/healthy/0402.jpg  
  inflating: /content/brain_tumor/healthy/0403.jpg  
  inflating: /content/brain_tumor/healthy/0404.jpg  
  inflating: /content/brain_tumor/healthy/0405.jpg  
  inflating: /content/brain_tumor/healthy/0406.jpg  
  inflating: /content/brain_tumor/healthy/0407.jpg  
  inflating: /content/brain_tumor/healthy/0408.jpg  
  inflating: /content/brain_tumor/healthy/0409.jpg  
  inflating: /content/brain_tumor/healthy/0410.jpg  
  inflating: /content/brain_tumor/healthy/0411.jpg  
  inflating: /content/brain_tumor/healthy/0412.jpg  
  inflating: /content/brain_tumor/healthy/0413.jpg  
  inflating: /content/brain_tumor/healthy/0414.jpg  
  inflating: /content/brain_tumor/healthy/0415.jpg  
  inflating: /content/brain_tumor/healthy/0416.jpg  
  inflating: /content/brain_tumor/healthy/0417.jpg  
  inflating: /content/brain_tumor/healthy/0418.jpg  
  inflating: /content/brain_tumor/healthy/0419.jpg  

In [6]:
!pip install torchvision --quiet
import os
from torchvision import transforms
from PIL import Image
import random


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.8 MB/s eta 0:00:00


📝 Note: Why is Data Augmentation Done?
Data augmentation is a critical step to increase the success of the model, especially in areas with limited data sets such as medical imaging. The purposes of data augmentation in this project are:

Reducing the risk of overfitting: Prevents the model from overfitting the training data and performing poorly on new images.

Increasing generalization ability: Makes the model more resistant to various image variations that may be encountered in the real world (rotation, brightness change, zooming, shifting, etc.).

Balancing imbalanced classes: If there are few images in some classes, it reduces class imbalance by creating more examples for these classes.

Increasing the accuracy of the model: A model trained with differentiated examples can predict with higher accuracy.

Reducing the cost of data collection: Generating derivatives from existing ones instead of collecting new images reduces the need for data.

In this project, 10 different variations were generated from each original MRI image, resulting in a much richer and more diverse training set per class, making our model both more robust and more generalizable.

In [7]:
# We define transformations for data augmentation
augmentation_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.7), # - Flip the image horizontally
    transforms.RandomRotation(30),  #- Rotate up to 30 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.4, saturation=0.4), #- Play with brightness, contrast and saturation
    transforms.RandomAffine(degrees=0, translate=(0.15, 0.15), scale=(0.85, 1.15)), # - Pan and zoom in/out the image
])


In [9]:
# Path of the original dataset (unaugmented)
source_path = "/content/brain_tumor"

# Temporary folder to store augmented data in /content
augmented_path = "/content/Brain_Tumor_Aug_dataset"
os.makedirs(augmented_path, exist_ok=True)

# Loop through each class folder (e.g., glioma, meningioma, etc.)
for class_name in os.listdir(source_path):
    class_dir = os.path.join(source_path, class_name)
    if not os.path.isdir(class_dir):
        continue  # Skip if not a directory

    new_class_dir = os.path.join(augmented_path, class_name)
    os.makedirs(new_class_dir, exist_ok=True)

    # Process each image in the class folder
    for img_name in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_name)
        try:
            image = Image.open(img_path).convert("RGB")  # Open and convert to RGB
        except:
            print(f"Skipping {img_path}")  # Skip corrupt or unreadable files
            continue

        # Save original image
        image.save(os.path.join(new_class_dir, img_name))

        # Create 10 augmented versions of the original image
        for i in range(10):
            augmented_image = augmentation_transforms(image)
            new_img_name = f"{os.path.splitext(img_name)[0]}_aug{i}.jpg"
            augmented_image.save(os.path.join(new_class_dir, new_img_name))


In [11]:
import zipfile
# Path to save the zip file temporarily
zip_path = "/content/Brain_Tumor_Aug_dataset.zip"

# Create the ZIP file
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(augmented_path):
        for file in files:
            full_path = os.path.join(root, file)
            rel_path = os.path.relpath(full_path, os.path.dirname(augmented_path))
            zipf.write(full_path, rel_path)

print("ZIP file successfully created at:", zip_path)


ZIP file successfully created at: /content/Brain_Tumor_Aug_dataset.zip


In [13]:
import shutil
# Destination path in Google Drive
drive_target = "/content/drive/MyDrive/brain_tumor/Brain_Tumor_Aug_dataset.zip"
shutil.copy(zip_path, drive_target)

print("ZIP file successfully copied to Drive at:", drive_target)


ZIP file successfully copied to Drive at: /content/drive/MyDrive/brain_tumor/Brain_Tumor_Aug_dataset.zip
